In [2]:
pip install requests


   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   -------- ------------------------------- 1/5 [idna]
   -------- ------------------------------- 1/5 [idna]
   -------- ------------------------------- 1/5 [idna]
   ---------------- ----------------------- 2/5 [charset_normalizer]
   ---------------- ----------------------- 2/5 [charset_normalizer]
   ---------------- ----------------------- 2/5 [charset_normalizer]
   ---------------- ----------------------- 2/5 [charset_normalizer]
   ------------------------ --------------- 3/5 [certifi]
   -------------------------------- 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# Base URL for the API
base_url_template = "https://bdl.stat.gov.pl/api/v1/data/by-variable/{variable_id}?format=xml&unit-level=3&page=0&page-size=100"

# Define the range of years
start_year = 2014
end_year = 2024

# Define variables for different animals (VALUE IN ZŁ)
animal_variables = {
    '151881': 'Jelenie',
    '151880': 'Sarny',
    '151879': 'Dziki'
}

all_animals_dfs = []

for var_id, animal_name in animal_variables.items():
    print(f"Fetching data for {animal_name} (variable ID: {var_id})")
    all_records_for_animal = []
    for year in range(start_year, end_year + 1):
        # Construct the URL for the current year and variable
        url = base_url_template.format(variable_id=var_id) + f"&year={year}"

        # Download XML data
        response = requests.get(url)
        response.raise_for_status()  # ensure request was successful
        xml_data = response.text

        # Parse the XML
        root = ET.fromstring(xml_data)

        # Extract relevant data
        for unit in root.findall(".//unitData"):
            name = unit.find("name").text
            val_element = unit.find(".//val")
            year_element = unit.find(".//year")

            val = float(val_element.text) if val_element is not None else None
            record_year = int(year_element.text) if year_element is not None else None

            all_records_for_animal.append({
                "Region": name,
                "Year": record_year,
                f"{animal_name} (tys zł)": val
            })
    
    df_animal = pd.DataFrame(all_records_for_animal)
    all_animals_dfs.append(df_animal)

# Merge all animal dataframes
# Start with the first dataframe and merge subsequent ones
df_merged = all_animals_dfs[0]
for i in range(1, len(all_animals_dfs)):
    df_merged = pd.merge(df_merged, all_animals_dfs[i], on=['Region', 'Year'], how='outer')

# Display or save
print(df_merged.head())
print(df_merged.tail())
df_merged.to_csv("bdl_data_2014-2024_all_animals_value.csv", index=False)

Fetching data for Jelenie (variable ID: 151881)
Fetching data for Sarny (variable ID: 151880)
Fetching data for Dziki (variable ID: 151879)
                Region  Year  Jelenie (tys zł)  Sarny (tys zł)  Dziki (tys zł)
0  REGION DOLNOŚLĄSKIE  2014            3564.1          3711.1          1439.4
1  REGION DOLNOŚLĄSKIE  2015            3847.4          3625.1          2648.0
2  REGION DOLNOŚLĄSKIE  2016            4915.7          4655.0          2974.6
3  REGION DOLNOŚLĄSKIE  2017            4875.6          3759.7          2678.4
4  REGION DOLNOŚLĄSKIE  2018            5493.7          3456.5          1289.2
                    Region  Year  Jelenie (tys zł)  Sarny (tys zł)  \
182  REGION ŚWIĘTOKRZYSKIE  2020             215.1           482.4   
183  REGION ŚWIĘTOKRZYSKIE  2021             217.2           560.8   
184  REGION ŚWIĘTOKRZYSKIE  2022            1218.2          1540.2   
185  REGION ŚWIĘTOKRZYSKIE  2023            1015.7          1746.4   
186  REGION ŚWIĘTOKRZYSKIE  2024    

In [18]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# Base URL for the API
base_url_template = "https://bdl.stat.gov.pl/api/v1/data/by-variable/{variable_id}?format=xml&unit-level=3&page=0&page-size=100"

# Define the range of years
start_year = 2014
end_year = 2024

# Define variables for different animals (QUANTITY IN TONS)
animal_variables = {
    '151877': 'Jelenie',
    '151884': 'Sarny',
    '151883': 'Dziki'
}

all_animals_dfs = []

for var_id, animal_name in animal_variables.items():
    print(f"Fetching data for {animal_name} (variable ID: {var_id})")
    all_records_for_animal = []
    for year in range(start_year, end_year + 1):
        # Construct the URL for the current year and variable
        url = base_url_template.format(variable_id=var_id) + f"&year={year}"

        # Download XML data
        response = requests.get(url)
        response.raise_for_status()  # ensure request was successful
        xml_data = response.text

        # Parse the XML
        root = ET.fromstring(xml_data)

        # Extract relevant data
        for unit in root.findall(".//unitData"):
            name = unit.find("name").text
            val_element = unit.find(".//val")
            year_element = unit.find(".//year")

            val = float(val_element.text) if val_element is not None else None
            record_year = int(year_element.text) if year_element is not None else None

            all_records_for_animal.append({
                "Region": name,
                "Year": record_year,
                f"{animal_name} (tony)": val
            })
    
    df_animal = pd.DataFrame(all_records_for_animal)
    all_animals_dfs.append(df_animal)

# Merge all animal dataframes
# Start with the first dataframe and merge subsequent ones
df_merged = all_animals_dfs[0]
for i in range(1, len(all_animals_dfs)):
    df_merged = pd.merge(df_merged, all_animals_dfs[i], on=['Region', 'Year'], how='outer')

# Display or save
print(df_merged.head())
print(df_merged.tail())
df_merged.to_csv("bdl_data_2014-2024_all_animals_tons.csv", index=False)

Fetching data for Jelenie (variable ID: 151877)
Fetching data for Sarny (variable ID: 151884)
Fetching data for Dziki (variable ID: 151883)
                Region  Year  Jelenie (tony)  Sarny (tony)  Dziki (tony)
0  REGION DOLNOŚLĄSKIE  2014           473.0         266.0         354.0
1  REGION DOLNOŚLĄSKIE  2015           466.0         247.0         548.0
2  REGION DOLNOŚLĄSKIE  2016           535.0         285.0         541.0
3  REGION DOLNOŚLĄSKIE  2017           504.0         271.0         620.0
4  REGION DOLNOŚLĄSKIE  2018           575.0         269.0         443.0
                    Region  Year  Jelenie (tony)  Sarny (tony)  Dziki (tony)
182  REGION ŚWIĘTOKRZYSKIE  2020            39.0          45.0           8.0
183  REGION ŚWIĘTOKRZYSKIE  2021            35.0          38.0          10.0
184  REGION ŚWIĘTOKRZYSKIE  2022           134.0          78.0          43.0
185  REGION ŚWIĘTOKRZYSKIE  2023           109.0          84.0          52.0
186  REGION ŚWIĘTOKRZYSKIE  2024     

In [19]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# Base URL for the API
base_url_template = "https://bdl.stat.gov.pl/api/v1/data/by-variable/{variable_id}?format=xml&unit-level=3&page=0&page-size=100"

# Define the range of years
start_year = 2014
end_year = 2024

# Define variables for different animals (CULLING/HUNTING IN PIECES)
animal_variables = {
    '4540': 'Jelenie',
    '4541': 'Sarny',
    '4542': 'Dziki'
}

all_animals_dfs = []

for var_id, animal_name in animal_variables.items():
    print(f"Fetching data for {animal_name} (variable ID: {var_id})")
    all_records_for_animal = []
    for year in range(start_year, end_year + 1):
        # Construct the URL for the current year and variable
        url = base_url_template.format(variable_id=var_id) + f"&year={year}"

        # Download XML data
        response = requests.get(url)
        response.raise_for_status()  # ensure request was successful
        xml_data = response.text

        # Parse the XML
        root = ET.fromstring(xml_data)

        # Extract relevant data
        for unit in root.findall(".//unitData"):
            name = unit.find("name").text
            val_element = unit.find(".//val")
            year_element = unit.find(".//year")

            val = float(val_element.text) if val_element is not None else None
            record_year = int(year_element.text) if year_element is not None else None

            all_records_for_animal.append({
                "Region": name,
                "Year": record_year,
                f"{animal_name} (odstrzał sztuki)": val
            })
    
    df_animal = pd.DataFrame(all_records_for_animal)
    all_animals_dfs.append(df_animal)

# Merge all animal dataframes
# Start with the first dataframe and merge subsequent ones
df_merged = all_animals_dfs[0]
for i in range(1, len(all_animals_dfs)):
    df_merged = pd.merge(df_merged, all_animals_dfs[i], on=['Region', 'Year'], how='outer')

# Display or save
print(df_merged.head())
print(df_merged.tail())
df_merged.to_csv("bdl_data_2014-2024_all_animals_hunted.csv", index=False)

Fetching data for Jelenie (variable ID: 4540)
Fetching data for Sarny (variable ID: 4541)
Fetching data for Dziki (variable ID: 4542)
                Region  Year  Jelenie (odstrzał sztuki)  \
0  REGION DOLNOŚLĄSKIE  2014                     7013.0   
1  REGION DOLNOŚLĄSKIE  2015                     7607.0   
2  REGION DOLNOŚLĄSKIE  2016                     7788.0   
3  REGION DOLNOŚLĄSKIE  2017                     8969.0   
4  REGION DOLNOŚLĄSKIE  2018                     9087.0   

   Sarny (odstrzał sztuki)  Dziki (odstrzał sztuki)  
0                  17973.0                  30210.0  
1                  18448.0                  38044.0  
2                  19808.0                  32381.0  
3                  20667.0                  39766.0  
4                  20239.0                  33844.0  
                    Region  Year  Jelenie (odstrzał sztuki)  \
182  REGION ŚWIĘTOKRZYSKIE  2020                     1194.0   
183  REGION ŚWIĘTOKRZYSKIE  2021                      388.0  

In [20]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# Base URL for the API
base_url_template = "https://bdl.stat.gov.pl/api/v1/data/by-variable/{variable_id}?format=xml&unit-level=3&page=0&page-size=100"

# Define the range of years
start_year = 2014
end_year = 2024

# Define variables for different animals (OVERALL AMOUNT)
animal_variables = {
    '4530': 'Jelenie',
    '4531': 'Sarny',
    '4532': 'Dziki'
}

all_animals_dfs = []

for var_id, animal_name in animal_variables.items():
    print(f"Fetching data for {animal_name} (variable ID: {var_id})")
    all_records_for_animal = []
    for year in range(start_year, end_year + 1):
        # Construct the URL for the current year and variable
        url = base_url_template.format(variable_id=var_id) + f"&year={year}"

        # Download XML data
        response = requests.get(url)
        response.raise_for_status()  # ensure request was successful
        xml_data = response.text

        # Parse the XML
        root = ET.fromstring(xml_data)

        # Extract relevant data
        for unit in root.findall(".//unitData"):
            name = unit.find("name").text
            val_element = unit.find(".//val")
            year_element = unit.find(".//year")

            val = float(val_element.text) if val_element is not None else None
            record_year = int(year_element.text) if year_element is not None else None

            all_records_for_animal.append({
                "Region": name,
                "Year": record_year,
                f"{animal_name} (ogółem tys. sztuk)": val
            })
    
    df_animal = pd.DataFrame(all_records_for_animal)
    all_animals_dfs.append(df_animal)

# Merge all animal dataframes
# Start with the first dataframe and merge subsequent ones
df_merged = all_animals_dfs[0]
for i in range(1, len(all_animals_dfs)):
    df_merged = pd.merge(df_merged, all_animals_dfs[i], on=['Region', 'Year'], how='outer')

# Display or save
print(df_merged.head())
print(df_merged.tail())
df_merged.to_csv("bdl_data_2014-2024_all_animals_overall_amount.csv", index=False)

Fetching data for Jelenie (variable ID: 4530)
Fetching data for Sarny (variable ID: 4531)
Fetching data for Dziki (variable ID: 4532)
                Region  Year  Jelenie (ogółem tys. sztuk)  \
0  REGION DOLNOŚLĄSKIE  2014                         17.1   
1  REGION DOLNOŚLĄSKIE  2015                         15.1   
2  REGION DOLNOŚLĄSKIE  2016                         15.6   
3  REGION DOLNOŚLĄSKIE  2017                         32.7   
4  REGION DOLNOŚLĄSKIE  2018                         29.4   

   Sarny (ogółem tys. sztuk)  Dziki (ogółem tys. sztuk)  
0                       83.9                       27.9  
1                       82.5                       23.1  
2                       85.1                       22.4  
3                       80.7                       29.9  
4                       79.0                       19.4  
                    Region  Year  Jelenie (ogółem tys. sztuk)  \
182  REGION ŚWIĘTOKRZYSKIE  2020                          4.5   
183  REGION ŚWIĘTOKRZ

In [21]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# Base URL for the API
base_url_template = "https://bdl.stat.gov.pl/api/v1/data/by-variable/{variable_id}?format=xml&unit-level=3&page=0&page-size=100"

# Define the range of years
start_year = 2014
end_year = 2024

# Define variables for different forest fire areas (EXPANSE IN HA)
fire_variables = {
    '151910': 'Powierzchnia pożarów lasów (ha)'
}

all_fire_dfs = []

for var_id, variable_name in fire_variables.items():
    print(f"Fetching data for {variable_name} (variable ID: {var_id})")
    all_records_for_variable = []
    for year in range(start_year, end_year + 1):
        # Construct the URL for the current year and variable
        url = base_url_template.format(variable_id=var_id) + f"&year={year}"

        # Download XML data
        response = requests.get(url)
        response.raise_for_status()  # ensure request was successful
        xml_data = response.text

        # Parse the XML
        root = ET.fromstring(xml_data)

        # Extract relevant data
        for unit in root.findall(".//unitData"):
            name = unit.find("name").text
            val_element = unit.find(".//val")
            year_element = unit.find(".//year")

            val = float(val_element.text) if val_element is not None else None
            record_year = int(year_element.text) if year_element is not None else None

            all_records_for_variable.append({
                "Region": name,
                "Year": record_year,
                f"{variable_name}": val
            })

    df_current_variable = pd.DataFrame(all_records_for_variable)
    all_fire_dfs.append(df_current_variable)

# Merge all dataframes
# Start with the first dataframe and merge subsequent ones
df_merged = all_fire_dfs[0]
for i in range(1, len(all_fire_dfs)):
    df_merged = pd.merge(df_merged, all_fire_dfs[i], on=['Region', 'Year'], how='outer')

# Display or save
print(df_merged.head())
print(df_merged.tail())
df_merged.to_csv("bdl_data_2014-2024_forest_fires_lands.csv", index=False)

Fetching data for Powierzchnia pożarów lasów (ha) (variable ID: 151910)
                      Region  Year  Powierzchnia pożarów lasów (ha)
0         REGION MAŁOPOLSKIE  2014                            71.46
1             REGION ŚLĄSKIE  2014                           205.22
2            REGION LUBUSKIE  2014                            30.91
3       REGION WIELKOPOLSKIE  2014                           118.41
4  REGION ZACHODNIOPOMORSKIE  2014                            43.34
                           Region  Year  Powierzchnia pożarów lasów (ha)
182              REGION LUBELSKIE  2024                            84.96
183           REGION PODKARPACKIE  2024                           111.69
184              REGION PODLASKIE  2024                            84.01
185   REGION WARSZAWSKI STOŁECZNY  2024                             0.00
186  REGION MAZOWIECKI REGIONALNY  2024                             0.00


In [22]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# Base URL for the API
base_url_template = "https://bdl.stat.gov.pl/api/v1/data/by-variable/{variable_id}?format=xml&unit-level=3&page=0&page-size=100"

# Define the range of years
start_year = 2014
end_year = 2024

# Define variables for forest land (EXPANSE IN HA)
forest_variables = {
    '1610493': 'Grunty zalesione',
    '1610500': 'Grunty niezalesione'
}

all_forest_dfs = []

for var_id, forest_land_name in forest_variables.items():
    print(f"Fetching data for {forest_land_name} (variable ID: {var_id})")
    all_records_for_forest = []
    for year in range(start_year, end_year + 1):
        # Construct the URL for the current year and variable
        url = base_url_template.format(variable_id=var_id) + f"&year={year}"

        # Download XML data
        response = requests.get(url)
        response.raise_for_status()  # ensure request was successful
        xml_data = response.text

        # Parse the XML
        root = ET.fromstring(xml_data)

        # Extract relevant data
        for unit in root.findall(".//unitData"):
            name = unit.find("name").text
            val_element = unit.find(".//val")
            year_element = unit.find(".//year")

            val = float(val_element.text) if val_element is not None else None
            record_year = int(year_element.text) if year_element is not None else None

            all_records_for_forest.append({
                "Region": name,
                "Year": record_year,
                f"Forest Lands - {forest_land_name} (ha)": val
            })

    df_forest = pd.DataFrame(all_records_for_forest)
    all_forest_dfs.append(df_forest)

# Merge all forest lands dataframes
# Start with the first dataframe and merge subsequent ones
df_merged = all_forest_dfs[0]
for i in range(1, len(all_forest_dfs)):
    df_merged = pd.merge(df_merged, all_forest_dfs[i], on=['Region', 'Year'], how='outer')

# Display or save
print(df_merged.head())
print(df_merged.tail())
df_merged.to_csv("bdl_data_2014-2024_forest_land_total.csv", index=False)

Fetching data for Grunty zalesione (variable ID: 1610493)
Fetching data for Grunty niezalesione (variable ID: 1610500)
                Region  Year  Forest Lands - Grunty zalesione (ha)  \
0  REGION DOLNOŚLĄSKIE  2017                             580912.31   
1  REGION DOLNOŚLĄSKIE  2018                             581220.72   
2  REGION DOLNOŚLĄSKIE  2019                             580812.35   
3  REGION DOLNOŚLĄSKIE  2020                             581456.97   
4  REGION DOLNOŚLĄSKIE  2021                             583111.79   

   Forest Lands - Grunty niezalesione (ha)  
0                                 13897.32  
1                                 13832.97  
2                                 14218.29  
3                                 14113.09  
4                                 13707.43  
                    Region  Year  Forest Lands - Grunty zalesione (ha)  \
131  REGION ŚWIĘTOKRZYSKIE  2020                             325284.75   
132  REGION ŚWIĘTOKRZYSKIE  2021          

In [23]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# Base URL for the API
base_url_template = "https://bdl.stat.gov.pl/api/v1/data/by-variable/{variable_id}?format=xml&unit-level=3&page=0&page-size=100"

# Define the range of years
start_year = 2014
end_year = 2024

# Define variables for forest area (EXPANSE IN HA)
forest_variables = {
    '217916': 'Powierzchnia lasów ogółem'
}

all_forest_dfs = []

for var_id, forest_name in forest_variables.items():
    print(f"Fetching data for {forest_name} (variable ID: {var_id})")
    all_records_for_forest = []
    for year in range(start_year, end_year + 1):
        # Construct the URL for the current year and variable
        url = base_url_template.format(variable_id=var_id) + f"&year={year}"

        # Download XML data
        response = requests.get(url)
        response.raise_for_status()  # ensure request was successful
        xml_data = response.text

        # Parse the XML
        root = ET.fromstring(xml_data)

        # Extract relevant data
        for unit in root.findall(".//unitData"):
            name = unit.find("name").text
            val_element = unit.find(".//val")
            year_element = unit.find(".//year")

            val = float(val_element.text) if val_element is not None else None
            record_year = int(year_element.text) if year_element is not None else None

            all_records_for_forest.append({
                "Region": name,
                "Year": record_year,
                f"{forest_name} (ha)": val
            })

df_forest = pd.DataFrame(all_records_for_forest)
all_forest_dfs.append(df_forest)

# Merge all forest dataframes
# Start with the first dataframe and merge subsequent ones
df_merged = all_forest_dfs[0]
for i in range(1, len(all_forest_dfs)):
    df_merged = pd.merge(df_merged, all_forest_dfs[i], on=['Region', 'Year'], how='outer')

# Display or save
print(df_merged.head())
print(df_merged.tail())
df_merged.to_csv("bdl_data_2014-2024_forest_area_total.csv", index=False)

Fetching data for Powierzchnia lasów ogółem (variable ID: 217916)
                      Region  Year  Powierzchnia lasów ogółem (ha)
0         REGION MAŁOPOLSKIE  2014                       435291.75
1             REGION ŚLĄSKIE  2014                       393849.16
2            REGION LUBUSKIE  2014                       688140.44
3       REGION WIELKOPOLSKIE  2014                       767530.76
4  REGION ZACHODNIOPOMORSKIE  2014                       811876.28
                           Region  Year  Powierzchnia lasów ogółem (ha)
182              REGION LUBELSKIE  2024                       589452.06
183           REGION PODKARPACKIE  2024                       685594.97
184              REGION PODLASKIE  2024                       631647.13
185   REGION WARSZAWSKI STOŁECZNY  2024                       140296.37
186  REGION MAZOWIECKI REGIONALNY  2024                       691592.04
